In [370]:
%run init_notebook.py

In [371]:
import eurostat
from src.utils import load_pd_df, get_dt_index, save_pd_df
from pandas_datareader.fred import FredReader

In [3]:
# interest rates

In [11]:
depo = FredReader('ECBDFR', start=datetime(2019,1,1), end=datetime.today()).read()
depo = depo.rename(columns={'ECBDFR': 'ecb_rate'}) / 100

In [12]:
df = pd.read_csv(os.path.join(DATA_DIR, 'ecb_interest_rates.csv'), low_memory=False)
df = df[df['REF_AREA'] == 'DE'].dropna(axis=1)
df['date'] = df['TIME_PERIOD'].apply(lambda x: datetime.strptime(x, "%Y-%m"))
df = df[df['TITLE'] == list(set(df['TITLE']))[1]]
df.set_index('date', inplace=True)

mir = df[['OBS_VALUE']].rename(columns={'OBS_VALUE': 'mir'}) / 100

In [13]:
mir = mir.join(depo)

In [14]:
mir = mir.ffill()

In [17]:
save_pd_df(mir, 'mir.csv')

In [10]:
# eurostat

In [11]:
codes = eurostat.get_toc_df()

In [12]:
[i for i in codes['title'] if 'deposit'.lower() in i.lower()]

['Loan and deposit one year interest rate']

In [13]:
filt = [
    'HICP - energy', 
    'HICP - food, alcohol and tobacco',
    'Gross domestic product (GDP) at current market prices by metropolitan regions',
    'Gross domestic product (GDP) at market prices - quarterly data',
    'HICP - inflation rate',
    'HICP - all items excluding energy and seasonal food',
    'HICP - energy',
    'HICP - monthly data (index)',
    'Final consumption expenditure',
    'Unemployment rate',
    'Unemployment rate by education level',
    'Unemployment rate (%) - monthly data',
    'Loan and deposit one year interest rate',
]
filt = [i in filt for i in codes['title']]

In [4]:
codes[filt];

In [10]:
# overall inflation

In [11]:
data = eurostat.get_data('med_ec2') #, filter_pars={'geo': ['DE','FR']})

In [11]:
df = pd.DataFrame(data[1:], columns=data[0]).transpose()
df = df.loc[:, (df.loc['unit'] == 'I05') & (df.loc['coicop'] == 'CP00')]
df.columns = [f'cpi_{i}' for i in df.loc['geo\TIME_PERIOD'].values]

cpi = df.iloc[5:].copy().dropna().astype(float)
cpi.index = [datetime(int(i[:4]), int(i[5:7]), 1) for i in cpi.index]

In [12]:
cpi['pi_de'] = np.log(cpi['cpi_DE']).diff()

In [13]:
save_pd_df(cpi, 'cpi.csv')

In [14]:
# food tobaco inflation

In [15]:
data = eurostat.get_data('TEICP260', filter_pars={'geo': ['DE']})
data = np.array(data)

In [16]:
df = pd.DataFrame(data=data[1:], columns=data[0])
df

,freq,coicop,unit,geo\TIME_PERIOD,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01
0,M,FOOD,I15,DE,139.5,141.3,141.1,141.1,141.2,141.2,141.0,141.5,141.6,142.4,142.4,None
1,M,FOOD,PCH_M1,DE,2.0,1.3,-0.1,0.0,0.1,0.0,-0.1,0.4,0.1,0.6,0.0,0.9
2,M,FOOD,PCH_M12,DE,18.0,18.6,15.5,13.6,12.6,10.7,9.2,8.1,7.1,6.6,5.8,5.1


In [3]:
# pi prognosis

In [4]:
df = get_dt_index(load_pd_df('ecb_prognosis_hicp_dec_2021.csv'), dt_index_col="DATE").iloc[:, 1:]
df.index.name = 'date'
df.columns = ["infl_forecast"]

In [5]:
df1 = load_pd_df('ecb_prognosis_hicp_jun_2021.csv')
df1['date'] = df1.DATE.apply(lambda x: datetime(int(x[:4]), 6, 30))
df1 = df1.drop('DATE', axis=1).set_index('date').iloc[:, 1:]
df1.columns = ["infl_forecast"]

df = pd.concat([df, df1], axis=0).sort_index()

In [19]:
save_pd_df(df, 'infl_prognonsis.csv')

# hhp

In [372]:
dfs = []
for i, file in enumerate(os.listdir(os.path.join(DATA_DIR, 'hhs_csv'))):
    try:
        d = pd.read_csv(os.path.join(DATA_DIR, 'hhs_csv', file), encoding='unicode_escape')
        dfs.append(d)
    except Exception as e:
        print(file, e)

In [373]:
# dfs, metadata = [], []
# for i, file in enumerate(os.listdir(os.path.join(DATA_DIR, 'hhs'))):
#     print(file)
#     try:
#         dta = pd.read_stata(os.path.join(DATA_DIR, 'hhs', file), )
#         dfs.append(dta)
#         metadata.append(metadata)
#     except Exception as e:
#         print(e)
#         continue

In [374]:
data = pd.concat(dfs, axis=0)

In [375]:
data['date_recorded'] = data[['year', 'month', 'day']].apply(lambda x: datetime(*x.values.astype(int)), axis=1).rename('date_recorded').values

C:\Users\LukasGrahl\AppData\Local\Temp\ipykernel_18244\3182224682.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['date_recorded'] = data[['year', 'month', 'day']].apply(lambda x: datetime(*x.values.astype(int)), axis=1).rename('date_recorded').values


In [376]:
dict_cols_rename = {'inflexppoint': 'pi_exp', 'inflexppoint_long': 'pi_exp_long', 'expint_sav': 'i_exp', 'devinfpoint': 'pi_perc'}

In [377]:
dict_cols_parse = {
    'id': float,
    # 'riskaversion': str,
    # 'date_recorded': 
    # 'date_forecast': 
    'i_exp': float,
    'pi_perc': float,
    'hhinc': str,
    'pinc': str,
    'pi_exp': float,
    'eduschool': str,
 'eduwork': str,
 'employ': str,
 'profession': str,
 'hhsize': str,
 'hhchildren': str,
 'familystatus': str,
    'ecbtrust': str,
 'hhinc': str,
 'pinc': str,
    'homeown': str,
    'pi_exp_long': float,
}
dict_cols_parse.update({f'infexprob_{i}': str for i in list(map(chr, range(97, 107)))})
dict_cols_parse.update({f'incexp_{i}': str for i in list(map(chr, range(97, 107)))})
dict_cols_parse.update({f'mainshopper_{i}': str for i in list(map(chr, range(97, 101)))})
dict_cols_parse.update({f'netwealth_detail_{i}': str for i in list(map(chr, range(97, 104)))})
dict_cols_parse.update({f'percprob_{i}': str for i in list(map(chr, range(97, 105)))})
# dict_cols_parse.update({f'mainshopper_{i}': str for i in list(map(chr, range(97, 1007)))})

lst_cols = list(dict_cols_parse.keys())
lst_cols.extend(['date_recorded'])

_ = [
        "don't know",
        'no answer',
        'does not apply',
        'nan',
    ]
dict_parse_nan = dict(zip(_, list([np.nan]) * len(_)))
dict_parse_nan_int = {
                '-9997.0': np.nan,
                  '-9998.0': np.nan,
                  '-9999.0': np.nan,
    '-9996.0': np.nan,
    '-6666.0': np.nan,                    
                     }

In [378]:
dict_inc_replace = {'3.500 to 3.999 EUR': '3.500 to 3.999 EUR',
 '2.000 to 2.499 EUR': '2.000 to 2.499 EUR',
 '500 to 999 EUR': '500 to 999 EUR',
 '3000 - < 3500 EUR': '3.000 to 3.499 EUR',
 '3500 - < 4000 EUR': '3.500 to 3.999 EUR',
 'under 500 EUR': 'under 500 EUR',
 '> 4499 EUR': '5.000 to 5.999 EUR',
 '1000 - < 1500 EUR': '1.000 to 1.499 EUR',
 '2000 - < 2500 EUR': '2.000 to 2.499 EUR',
 np.nan: np.nan,
 '< 500 EUR': 'under 500 EUR',
 '3.000 to 3.499 EUR': '3.000 to 3.499 EUR',
 '6.000 to 7.999 EUR': '6.000 to 7.999 EUR',
 '4000 - < 4500 EUR': '4.000 to 4.999 EUR',
 '4.000 to 4.999 EUR': '4.000 to 4.999 EUR',
 '5.000 to 5.999 EUR': '5.000 to 5.999 EUR',
 '10.000 EUR and more': '10.000 EUR and more',
 '6.000 EUR and more': '6.000 to 7.999 EUR',
 '2500 - < 3000 EUR': '2.500 to 2.999 EUR',
 '8.000 to 9.999 EUR': '8.000 to 9.999 EUR',
 '1500 - < 2000 EUR': '1.500 to 1.999 EUR',
 '1.500 to 1.999 EUR': '1.500 to 1.999 EUR',
 '500 - < 1000 EUR': '500 to 999 EUR',
  '500 bis 999 EUR': '500 to 999 EUR',
 '2.500 to 2.999 EUR': '2.500 to 2.999 EUR',
 '1.000 to 1.499 EUR': '1.000 to 1.499 EUR'}
dict_inc_replace.update(dict_parse_nan)

In [379]:
data = data.rename(columns=dict_cols_rename)

# df

In [448]:
def get_var_time_delta(df_in: pd.DataFrame, col: str, index: list = ['id', 'date_recorded']):
    sub = df_in.sort_values(index[1]).set_index(index)[col].unstack().copy().astype(float) + 1
    sub = (sub.ffill(axis=1).diff(axis=1).stack()).rename(f'{col}_delta')
    return df_in.set_index(index).join(sub).reset_index()

In [449]:
df = data[lst_cols].copy()
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [450]:
df['date_forecast'] = df.date_recorded + timedelta(days=365)

In [451]:
# inflation

In [452]:
cols = [i for i in dict_cols_parse if sum([term in i for term in ['infexprob', 'incexp']]) > 0]
for col in cols:
    df[col] = df[col].apply(lambda x: dict_parse_nan_int[str(x)] if str(x) in dict_parse_nan_int.keys() else x)

In [453]:
# parse nans

In [465]:
for col in ['pi_exp', 'pi_perc', 'i_exp', 'pi_exp_long',]:
    df = df.drop(col, axis=1).join(df[col].apply(lambda x: dict_parse_nan_int[str(x)] if str(x) in dict_parse_nan_int.keys() else x))

In [387]:
for col in ['profession', 'ecbtrust', 'hhsize', 'homeown']:
    df[col] = df[col].astype(float).apply(lambda x: dict_parse_nan_int[str(x)] if str(x) in dict_parse_nan_int.keys() else x)

In [388]:
for col in ['hhinc', 'pinc']:
    df[col] = df[col].astype(float).apply(lambda x: dict_parse_nan_int[str(x)] if str(x) in dict_parse_nan_int.keys() else x)

In [389]:
dict_profession_int_txt = {
    1: "Arbeiter(in), auch in der Landwirtschaft",
2: "Angestellte(r)",
3: "Beamtin / Beamter, einschl. Richter / Richterin und Berufssoldaten / Berufssoldatinnen / Zeitsoldaten / Zeitsoldatinnen",
4: "Selbständige(r) oder Unternehmer (in) (einschl. selbständiger Landwirte / Landwirtinnen) ohne abhängig Beschäftigte",
5: "Selbständige(r) oder Unternehmer (in) (einschl. selbständiger Landwirte / Landwirtinnen) mit abhängig Beschäftigten",
6: "Auszubildende(r) / Praktikant(in)",
7: "Unbezahlte(r) mithelfende(r) Familienangehörige(r)",
8: "Sonstiges"
}

In [390]:
df['profession_txt'] = df['profession'].map(dict_profession_int_txt)

In [391]:
for col in [
    *[i for i in df.columns if 'mainshopper' in i],
    *[i for i in df.columns if 'netwealth' in i],
    *[i for i in df.columns if 'percprob' in i]
]:
    df[col] = df[col].astype(float).apply(lambda x: dict_parse_nan_int[str(x)] if str(x) in dict_parse_nan_int.keys() else x)

In [392]:
# ecb trust

In [393]:
dict_ecb_trust = {
    "do not trust at all": 0,
    "0 do not trust at all": 0,
    "I do not know the ECB": 0,
    "fully trust": 10,
    "10 trust completely": 10,
    "ECB unknown": 0,    
    "trust completely": 10
}

In [394]:
df = get_var_time_delta(df, 'ecbtrust')

In [395]:
# income

In [396]:
dict_hhinc_int_midpoint = {
    1: 250, # "unter 500 Euro",
    2: 750, # "500 bis 999 Euro",
    3: 1250, #"1000 bis 1.499 Euro",
    4: 1750, #"1.500 bis 1.999 Euro",
    5: 2250, #"2.000 bis 2.499 Euro",
    6: 2750, #"2.500 bis 2.999 Euro",
    7: 3250, #"3.000 bis 3.499 Euro",
    8: 3750, #"3.500 bis 3.999 Euro",
    9: 4250, #"4.000 bis 4.999 Euro",
    10: 5500, #"5.000 bis 5.999 Euro",
    11: 7000, #"6.000 bis 7.999 Euro",
    12: 9000, #"8.000 bis 9.999 Euro",
    13: 12000, #"10.000 Euro und mehr",
}
dict_pinc_int_midpoint = {
    1: 250, #unter 500 Euro
2: 750, # 500 bis 999 Euro
3: 1250, # 1000 bis 1.499 Euro
4: 1750, # 1.500 bis 1.999 Euro
5: 2250, # 2.000 bis 2.499 Euro
6: 2750, # 2.500 bis 2.999 Euro
7: 3250, # 3.000 bis 3.499 Euro
8: 3750, # 3.500 bis 3.999 Euro
9: 4500, # 4.000 bis 4.999 Euro
10: 5500, # 5.000 bis 5.999 Euro
11: 7000, # 6.000 Euro und mehr
}

In [397]:
df['hhinc_midpoint'] = df['hhinc'].map(dict_hhinc_int_midpoint)

In [398]:
df['pinc_midpoint'] = df['pinc'].map(dict_pinc_int_midpoint)

In [399]:
# df['pinc_midpoint'] = df['pinc'].map(dict_hhinc_int_midpoint)
df = df.sort_values('date_recorded')
sub = df.set_index(['id', 'date_recorded',])['pinc_midpoint'].unstack()
sub = sub.ffill(axis=1).diff(axis=1).stack().rename('pinc_delta')
df = df.set_index(['id', 'date_recorded']).join(sub).reset_index()

In [400]:
# df['hhinc_midpoint'] = df['hhinc'].map(dict_hhinc_midpoint)
df = df.sort_values('date_recorded')
sub = df.set_index(['id', 'date_recorded',])['hhinc_midpoint'].unstack()
sub = sub.ffill(axis=1).diff(axis=1).stack().rename('hhinc_delta')
df = df.set_index(['id', 'date_recorded']).join(sub).reset_index()

In [401]:
# employement educ

In [402]:
dict_eduschool_int_txt = {
    1: "derzeit noch Schüler / Schülerin",
2: "Hauptschule / Volksschulabschluss",
3: "Mittlere Reife / Realschulabschluss",
4: "Abschluss Polytechnische Oberschule, 10. Klasse",
5: "Fachhochschulreife, Abschluss einer Fachoberschule",
6: "Allgemeine oder fachgebundene Hochschulreife / Abitur (Gymnasium bzw. EOS auch EOS mit Lehre)",
7: "anderer Schulabschluss",
8: "kein Schulabschluss (und aktuell kein Schüler / keine Schülerin)"
}

In [403]:
dict_eduwork_int_txt = {
    1: "derzeit in Ausbildung bzw. Studium (noch keinen Bachelorabschluss erreicht)",
2: "beruflich-betriebliche Ausbildung (Lehre) abgeschlossen",
3: "beruflich-schulische Ausbildung (Berufsfachschule, Höhere Handelsschule) abgeschlossen",
4: "Ausbildung an einer Fachschule, Technikerschule, Berufs- oder Fachakademie abgeschlossen, Meisterschule mit kurzer Vorbereitungszeit (bis zu 880: Stunden) abgeschlossen",
5: "Abschluss einer Meisterschule mit langer Vorbereitungszeit von mehr als 880: Stunden",
6: "Bachelor, Fachhochschulabschluss, Ingenieurschule abgeschlossen",
7: "Diplom- oder Masterabschluss, Lehrerausbildung abgeschlossen",
8: "Promotion / Habilitation abgeschlossen",
9: "anderer beruflicher Abschluss",
10: "kein Ausbildungsabschluss (und aktuell nicht in Ausbildung / im Studium)"
}

In [404]:
dict_employ_int_txt = {
    1: "Erwerbstätig (auch Lehre) in Vollzeit",
2: "Erwerbstätig Teilzeit (auch Altersteilzeit)",
3: "geringfügig oder unregelmäßig beschäftigt (auch bezahltes Praktikum sowie Eingliederungsmaßnahmen)",
4: "in Mutterschutz / in Elternzeit / länger krankgeschrieben / sonstige Beurlaubung. Die Rückkehr zurErwerbstätigkeit ist geplant",
5: "Arbeitslos (offiziell gemeldet)",
6: "in der Schule, Studium oder unbezahltes Praktikum",
7: "Rentner(in) oder Pensionär(in)",
8: "Frührentner(in) oder Vorruheständler(in) (auch Arbeitsunfähige, Berufsunfähige, Erwerbsunfähige oder -geminderte, Dienstunfähige)",
9: "Bundesfreiwilligendienst / Freiwilliges Jahr",
10: "Hausfrau / Hausmann",
11: "sonstige Nichterwerbstätigkeit",
}


In [405]:
df['eduschool_txt'] = df['eduschool'].map(dict_eduschool_int_txt)
df['eduwork_txt'] = df['eduwork'].map(dict_eduwork_int_txt)
df['employ_txt'] = df['employ'].map(dict_employ_int_txt)

In [406]:
df['is_unempl'] = df['employ'] == 5

In [407]:
df = get_var_time_delta(df, 'is_unempl')
df['is_unempl_delta'] = df.is_unempl_delta * (-1)

In [408]:
# a-d cats

In [409]:
df['is_food_shop'] = ~(df['mainshopper_a'] == 3)
df['is_invest_shop'] = ~(df['mainshopper_b'] == 3)
df['is_invest_manag'] = ~(df['mainshopper_d'] == 3)

In [410]:
# wealth

In [411]:
df['wealth_bank'] = df['netwealth_detail_a'] #map(dict_networth_mid_point)
df['debt'] = df['netwealth_detail_g'] #.map(dict_debt_midpoint)

In [412]:
df = get_var_time_delta(df, 'debt')
df = get_var_time_delta(df, 'wealth_bank')

In [413]:
# homeown

In [415]:
df['is_homeown'] = df['homeown'] != 1

In [416]:
df = get_var_time_delta(df, 'is_homeown')
df['is_homeown_delta'] = df.is_homeown_delta * (-1)

In [417]:
# perceived problems

In [418]:
df['percprob_covid'] = df['percprob_b']
df['percprob_reces'] = df['percprob_d']
df['percprob_war'] = df['percprob_e']

In [419]:
# save

In [467]:
df[['id', 'date_recorded', 'pi_perc']].drop_duplicates().dropna().groupby('id').pi_perc.count().describe()

count    45440.000000
mean         2.328433
std          1.878436
min          1.000000
25%          1.000000
50%          1.000000
75%          3.000000
max         10.000000
Name: pi_perc, dtype: float64

In [368]:
df = df.astype({k: v for k, v in dict_cols_parse.items() if k in df.columns})
df = df.replace({'nan': np.nan})

In [369]:
df.to_feather(os.path.join(DATA_DIR, 'hhp.feather'))